In [1]:
import pycuda
import pycuda.driver as cuda
import pycuda.autoinit

from pycuda.compiler import SourceModule

import numpy as np
import time

print(pycuda.VERSION)

"""
Matrix multiplication example
"""

(2019, 1, 1)


'\nMatrix multiplication example\n'

In [2]:
MAT_SIZE = (16, 16) # row, col
BLOCKSIZE = 4
GRIDSIZE = int(np.ceil(256/BLOCKSIZE))
THREAD_PER_BLOCK = 4

In [3]:
# Create a tensor and copy it to gpu memory
# Randomly initialize a vector
#a = np.ones(shape=(64*64), dtype=np.float32)
a = np.ones(shape=MAT_SIZE, dtype=np.float32)
a = a.astype(np.float32)
b = np.ones(shape=MAT_SIZE, dtype=np.float32)
# Allocate memory at device
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
# Allocate same size of memory for result vector
result = np.zeros(shape=MAT_SIZE, dtype=np.float32)
result_gpu = cuda.mem_alloc(result.nbytes)
# Copy the cpu vector to gpu
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)
# width and height
w = np.array([16], dtype=np.int)
h = np.array([16], dtype=np.int)
width = cuda.mem_alloc(w.nbytes)
height = cuda.mem_alloc(h.nbytes)

cuda.memcpy_htod(width, w)
cuda.memcpy_htod(height, h)

In [4]:
mod = SourceModule("""
    __global__ void matMul2D(float *a, float *b, float *res, int *width, int *height)
    {
        int tx = blockDim.x * blockIdx.x + threadIdx.x;
        int ty = blockDim.y * blockIdx.y + threadIdx.y;
        int tid = width[0] * ty + tx;
        
        int value = 0;
        int Mval = 0;
        int Nval = 0;
        
        for (int i=0; i<width[0]; i++)
        {
            Mval = a[ty*width[0]+i];
            Nval = b[i*width[0]+tx];
            value += Mval * Nval;
        }
        
        res[tid] = value;
    }
""")

In [5]:
# Let's compare operating time
# GPU
startTime = time.time()
func = mod.get_function("matMul2D")
func(a_gpu, b_gpu, result_gpu, width, height, block=(16,16,1), grid =(1,1))
consumedTime = time.time() - startTime
print("Time for gpu operation : ", consumedTime)


Time for gpu operation :  0.0006635189056396484


In [6]:
# CPU
startTime = time.time()
result_cpu = np.dot(a, b)
consumedTime = time.time() - startTime
print("Time for cpu operation : ", consumedTime)

Time for cpu operation :  0.0008044242858886719


In [7]:
# Copy the result from device to host
#result = np.zeros(shape=(4), dtype=np.float32)
cuda.memcpy_dtoh(result, result_gpu)
a_gpu.free()
b_gpu.free()
result_gpu.free()
width.free()
height.free()

In [8]:
# Compare the results
print("Is it same? : ", (result == result_cpu).all())

Is it same? :  True


In [9]:
result_cpu

array([[16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16

In [10]:
result

array([[16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16., 16.,
        16., 16., 16.],
       [16., 16., 16., 16., 16